In [3]:
#シンチがとったコインシデンスの数/1s
import pandas as pd
sf = pd.read_csv('angel_data.csv', header=None)
df = pd.read_csv('curve_plot.csv', header=None)
num = 1

time = float(sf.at[num, 3])
minute = time/60
toukei = [int(sf.at[num, 8+i]) for i in range(5)]

In [4]:
#overlapのシミュレーション
import numpy as np
N = 10000
overlap_minute = [[] for _ in range(5)]

for i in range(5): #シンチのペアごとに
    for j in range(N): #N回シミュレーションして、
        duration = 60  #秒、1分　<- ここを長くしすぎるととんでもない時間かかるので、1分にして後で直す
        n_a = int(toukei[i]/minute)       # 信号Aの数
        n_b = int(toukei[i]/minute)       # 信号Bの数
        width_a = 100e-6   # パルスAの幅（秒）
        width_b = 100e-6     # パルスBの幅（秒）

        # 信号の開始時刻をランダムにサンプリング
        start_times_a = np.random.uniform(0, duration, n_a)
        start_times_b = np.random.uniform(0, duration, n_b)

        # パルスの終了時刻を計算
        end_times_a = start_times_a + width_a
        end_times_b = start_times_b + width_b

        # コインシデンスのカウント
        coincidence_count = 0
        for k in range(n_a):
            a_start = start_times_a[k]
            a_end = end_times_a[k]
            
            # B信号との重なりをチェック
            overlaps = np.logical_and(start_times_b < a_end, end_times_b > a_start)
            coincidence_count += np.count_nonzero(overlaps)
        overlap_minute[i].append(coincidence_count)

In [5]:
#overlapの誤差の計算
overlap_time = [np.average(overlap_minute[i])*minute for i in range(5)]
print(overlap_time)

error5_overlap = [float(e/d) for e,d in zip(overlap_time, toukei)]
print(error5_overlap)

[np.float64(0.024), np.float64(2.1599999999999997), np.float64(0.288), np.float64(0.624), np.float64(0.048)]
[2.25140712945591e-05, 0.00017136057120190399, 6.036470341647453e-05, 0.00010436527847466132, 1.5213946117274169e-05]


In [7]:
#errorの保存
for i in range(5):
    df.at[num, i+47] = error5_overlap[i]
df.to_csv('curve_plot.csv', index=False, header=False)